In [1]:
import cv2 # импорт библтотеки для работы с изображениями
import numpy as np  # имппорт библиотеки мат.операций numpy. сокращеют ее до NP
from matplotlib import pyplot
from IPython import display
from IPython.display import Image

In [4]:
# организуем нейросеть
net = cv2.dnn.readNet('yolo/yolov3.weights', 'yolo/yolov3.cfg')

# из библиотеки CV2, берем модуль dnn, даем команду readnet(), которая из файла читает веса и конфигурацию нашей нйеросети

In [5]:
# читаем файл с названиями классов, которые распознает нейросеть
with open('yolo/coco.names.txt') as f:
    labels = f.read().strip().split('\n')
    
        
    # split делает построчное представление
    # strip  отрезает последнюю строку

In [6]:
len(labels)

80

In [7]:


img = cv2.imread('image1.jpg') # считали картинку
height, width, _ = img.shape  # из картинки считываем длину ширину и кладем в соответствующие переменные
blob = cv2.dnn.blobFromImage(img, 1/255, (608, 608), (0, 0, 0), swapRB=True)
net.setInput(blob)
#blob = cv2.dnn.blobFromImage(img, 1/255, (608,608), (0,0,0), swapRB=True)
# вспомогательная функция blobFromImage
# обьект img
# 1/255 коэффициент, на который надо домножить каждый пиксель
# (608,608) размеры фреймов, с которыми будет работать нейросеть. надо смотреть в файле конфигурации yolov3.cfg. строки 9,8. 
#  (0,0,0)  ничего не вычитать из цветов
#  swapRB=True  меняем RGB на BGR
#
# net.setInput(blob)  функция setInput кладет  blob в  net

In [ ]:


layer_names = net.getLayerNames()   # вынимаем наменования слоев из сети в 
out_layers_indexes_arr = net.getUnconnectedOutLayers()
out_layers_indexes = [index[0] - 1 for index in out_layers_indexes_arr]
out_layer_names = [layer_names[index] for index in out_layers_indexes]
out_layer_names

In [ ]:
out_layers = net.forward(out_layer_names)

In [ ]:
#  готовим функцию, которая рисует прямоугольники и названия на них (см ноутбук Day1)
# label  это тип класса, для которого найдены вероятности

def draw_object(img, x, y, w, h, label): # на вход подаем значения - картинка, начало фрейма(координаты левого верхнего угла)
    x1, y1 = x - w // 2, y - h // 2  # первая координата, левого верхнего угла прямоугольника
    x2, y2 = x + w // 2, y + h // 2 # вторая координата, правого нижнего угла прямоугольника
    color = [0, 255, 0] # цвет
    width = 5           # толщина линии

    img = cv2.rectangle(img, (x1, y1), (x2, y2), color, width)  # рисуем прямоугольник

    font_size = 3 #размер шрифта
    font = cv2.FONT_HERSHEY_SIMPLEX  # применяемый шрифт
    text = label # текстом будет класс обьекта с найденной вероятностью

    img = cv2.putText(img, text, (x1, y1 - 10), font, font_size, color, width) # пишем слово
    
    return img

In [ ]:
object_boxes = []  #формируем список боксов
object_probas = []# вероятностей
object_labels = []# названий классов

for layer in out_layers:
    for result in layer:
        x, y, w, h = result[:4] # x y w h - им присваиваем первые 4 параметра из result 
        x = int(x * width) # воссттанавливаем первоначальное значение (координату) центра фрейма (пикселя) (от левого верхнего угла по горизонтали), INT - отбрасываем до целого числа 
        w = int(w * width)# воссттанавливаем первоначальное значение (координату) центра фрейма (пикселя) (от левого верхнего угла по вертикали)
        y = int(y * height) #воссттанавливаем первоначальное значение фрейма (его ширина)
        h = int(h * height)# воссттанавливаем первоначальное значение фрейма (его высота)
        probas = result[5:] # после пятого элемента в RESULT- это вероятность
        max_proba_index = np.argmax(probas) # ищем индекс максимального элемента среди всех вероятностей
        max_proba = probas[max_proba_index] # берем список вероятностей и вынимаем из него элемент с указанным индексом
        if max_proba > 0: # !!!!!!  если вероятность max_proba больше нуля, то
            
            
            
            object_boxes.append([x, y, w, h]) # то в массив object_boxes добавляем x, y, w, h
            object_probas.append(float(max_proba)) # представление числа из Йола в Пайтоновский вид
            object_labels.append(labels[max_proba_index])  # находим лейбл (название класса обьектм) через индекс найденной вероятности

In [ ]:
# Фильтруем пересекающиеся прямоугольники (по индексам)

filtered_boxes_indexes = cv2.dnn.NMSBoxes(object_boxes, object_probas, 0.0, 0.3)

# фильтр. отфильтруем  изображения через алгоритм NMSBoxes
# принимает на вход список обьектов Boxes
# список вероятностей по всем нашим обьектам
#  веро]тность, начиная с которой можно вообще начинать рисовать
#  процент пересечения площадей !!!!!!!
#++++++++++++++++++++++++++++++


In [ ]:
img = cv2.imread('image1.jpg')
for index_arr in filtered_boxes_indexes:
    index = index_arr[0]
    box = object_boxes[index]
    x, y, w, h = box
    img = draw_object(img, x, y, w, h, object_labels[index])

In [ ]:
cv2.imwrite('image1_test.jpg', img)  # записываем изображение img (со всеми нарисованными элементами) в другой файл на диск
Image('image1_test.jpg') # показать файл

In [ ]:
with open('yolo/coco.names.txt') as f:
    labels = f.read().strip().split('\n')

layer_names = net.getLayerNames()
out_layers_indexes_arr = net.getUnconnectedOutLayers()
out_layers_indexes = [index[0] - 1 for index in out_layers_indexes_arr]
out_layer_names = [layer_names[index] for index in out_layers_indexes]
out_layer_names

In [ ]:
def draw_object(img, x, y, w, h, label):
    x1, y1 = x - w // 2, y - h // 2
    x2, y2 = x + w // 2, y + h // 2
    color = [0, 255, 0]
    width = 3

    img = cv2.rectangle(img, (x1, y1), (x2, y2), color, width)

    font_size = 1
    font = cv2.FONT_HERSHEY_SIMPLEX
    text = label

    img = cv2.putText(img, text, (x1, y1 - 10), font, font_size, color, width)
    
    return img

def apply_yolo(img):
    height, width, _ = img.shape
    blob = cv2.dnn.blobFromImage(img, 1/255, (608, 608), (0, 0, 0), swapRB=True)
    net.setInput(blob)
    
    out_layers = net.forward(out_layer_names)
    
    object_boxes = []
    object_probas = []
    object_labels = []

    for layer in out_layers:
        for result in layer:
            x, y, w, h = result[:4]
            x = int(x * width)
            w = int(w * width)
            y = int(y * height)
            h = int(h * height)
            probas = result[5:]
            max_proba_index = np.argmax(probas)
            max_proba = probas[max_proba_index]
            if max_proba > 0:
                object_boxes.append([x, y, w, h])
                object_probas.append(float(max_proba))
                object_labels.append(labels[max_proba_index])

    filtered_boxes_indexes = cv2.dnn.NMSBoxes(object_boxes, object_probas, 0.0, 0.3)
    
    for index_arr in filtered_boxes_indexes:
        index = index_arr[0]
        box = object_boxes[index]
        x, y, w, h = box
        img = draw_object(img, x, y, w, h, object_labels[index])
    
    return img

In [ ]:
cap = cv2.VideoCapture('video.mp4')

while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        break

    frame = apply_yolo(frame)
    
    frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)    
    
    pyplot.figure(figsize=(20, 15))
    pyplot.imshow(frame)
    display.clear_output(wait=True)
    display.display(pyplot.gcf())
    

cap.release()

In [ ]:
size = (2560 // 2, 1440 // 2)

codec = cv2.VideoWriter_fourcc(*'mp4v')
writer = cv2.VideoWriter('output.mp4', codec, 25, size)
# codec = cv2.VideoWriter_fourcc(*'MJPG')
# out = cv2.VideoWriter('output.avi', codec, 25, size)

cap = cv2.VideoCapture('video.mp4')

#while cap.isOpened():
for i in range(20):
    print('.', end='')

    ret, frame = cap.read()
    if not ret:
        break

    frame = apply_yolo(frame)
    frame = cv2.resize(frame, size)
    writer.write(frame)

cap.release()
writer.release()